# import

In [99]:
import requests
from math import radians, cos, sin, asin, sqrt

# google map api

In [65]:
f = open("API_KEY.txt", "r")
API_KEY = f.read()

In [66]:
def get_location_information(address,API_KEY,language="zh-TW"):
    req_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    my_params = {
        "address" : address,
        "key" : API_KEY ,
        "language" : language
    }
    r = requests.get(req_url,params=my_params)
    res = r.json()
    data = {}
    if(res['status']=="OK"):
        data['address'] = res['results'][0]['formatted_address']
        data['geocode'] = res['results'][0]['geometry']['location']
        data['place_id'] = res['results'][0]['place_id']
    elif(res['status']=='ZERO_RESULTS'):
        data['message'] = "zero results"
    elif(res['status']=='INVALID_REQUEST') :
        data['message'] = res['error_message']
    return {"status":res['status'],"data":data}

In [69]:
get_location_information("永寧捷運站",API_KEY)

{'data': {'address': '236台灣新北市土城區中央路3段105號永寧站',
  'geocode': {'lat': 24.9669358, 'lng': 121.4362613},
  'place_id': 'ChIJY1sYMdccaDQRhP5havdXk2Q'},
 'status': 'OK'}

# get all MRT station

In [70]:
r = requests.get('https://gist.githubusercontent.com/lackneets/f41836e7e4e938a89f73/raw/e427e8c7b58fcce1338c96bc38b890ae2db6a98a/mrt-taipei.json')

In [74]:
mrt_data = r.json()

In [75]:
mrt_data

[{'address': '115台灣台北市南港區捷運南港軟體園區站',
  'id': '030',
  'latitude': 25.059904,
  'line': '文山內湖線',
  'longitude': 121.615952,
  'name': '南港軟體園區'},
 {'address': '115台灣台北市南港區捷運南港展覽館站',
  'id': '031',
  'latitude': 25.055368,
  'line': '文山內湖線',
  'longitude': 121.6176,
  'name': '南港展覽館'},
 {'address': '114台灣台北市內湖區捷運東湖站',
  'id': '029',
  'latitude': 25.067247,
  'line': '文山內湖線',
  'longitude': 121.611504,
  'name': '東湖'},
 {'address': '114台灣台北市內湖區捷運葫洲站',
  'id': '028',
  'latitude': 25.072701,
  'line': '文山內湖線',
  'longitude': 121.607158,
  'name': '葫洲'},
 {'address': '114台灣台北市內湖區捷運大湖公園站',
  'id': '027',
  'latitude': 25.083844,
  'line': '文山內湖線',
  'longitude': 121.60214,
  'name': '大湖公園'},
 {'address': '114台灣台北市內湖區捷運內湖站',
  'id': '026',
  'latitude': 25.08366,
  'line': '文山內湖線',
  'longitude': 121.594407,
  'name': '內湖'},
 {'address': '114台灣台北市內湖區捷運文德站',
  'id': '025',
  'latitude': 25.078531,
  'line': '文山內湖線',
  'longitude': 121.584761,
  'name': '文德'},
 {'address': '114台灣台北市內湖區捷運港墘站',
 

# calculate distance with geocode

In [98]:
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

In [79]:
haversine(121.4362613,24.9669358,121.567213,25.082133)

18389.336632420665

In [96]:
def distance_google(origin,destination,API_KEY,language='zh-TW'):
    map_url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"
    map_params = {
        "origins" : origin,
        "destinations" : destination,
        "key" : API_KEY , 
        "language" : language
    }
    r = requests.get(map_url,map_params)
    res = r.json()
    return r.json()['rows'][0]['elements'][0]

In [97]:
distance_google("霧峰區大同路52巷","永寧捷運站",API_KEY)

{'distance': {'text': '108 哩', 'value': 173069},
 'duration': {'text': '1 小時 47 分', 'value': 6440},
 'status': 'OK'}